In [31]:
"""
In this notebook, I try usage of 
logsoftmax and crossentropy
I dump all inputs in all steps.
At the end, i tried manual calculation of crossentropy.
"""
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn
from torch.autograd import Variable
from math import exp
import math


In [12]:
class SimpleModel(nn.Module):

    def __init__(self, n_features, n_hiddens, n_classes,apply_transform):
        super(SimpleModel, self).__init__()
        self.gru = torch.nn.GRU(n_features, n_hiddens)
        self.linear = torch.nn.Linear(n_hiddens, n_classes)
        self.apply_transform = apply_transform

    def forward(self, x):
        o, h = self.gru(x)
        o = self.linear(o)

        if self.apply_transform == 1:
            o = F.log_softmax(o)


        return o

In [13]:
n_steps = 2 #sequence
n_classes = 4 #target class label dimension
mb_size = 3 #batch
n_features = 5 #input_size 
n_hiddens = 6 #hidden size
#(gru): GRU(5, 6)
#(linear): Linear(in_features=6, out_features=4, bias=True)

In [14]:
nll_loss = torch.nn.NLLLoss(size_average=False)
cross_entropy = torch.nn.CrossEntropyLoss(size_average=False)

x = Variable(torch.rand(n_steps, mb_size, n_features))
y = Variable(
    torch.LongTensor(np.random.randint(0, n_classes, (n_steps, mb_size))))

/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [15]:
print("x",x.size())
print("x",x)
print("y",y.size())
print("y",y)
#2 steps
#3 batch size
#5 number of features

x torch.Size([2, 3, 5])
x tensor([[[0.3080, 0.5794, 0.4141, 0.9090, 0.4658],
         [0.6117, 0.1690, 0.9558, 0.3141, 0.5847],
         [0.4494, 0.7352, 0.6705, 0.1690, 0.7494]],

        [[0.6088, 0.6791, 0.8987, 0.8343, 0.8775],
         [0.5871, 0.5587, 0.8751, 0.7659, 0.1382],
         [0.9957, 0.3977, 0.6809, 0.3777, 0.6591]]])
y torch.Size([2, 3])
y tensor([[2, 0, 2],
        [0, 0, 0]])


In [16]:
model_cross = SimpleModel(n_features, n_hiddens, n_classes,apply_transform=0)
model_logsoftmax = SimpleModel(n_features, n_hiddens, n_classes,apply_transform=1)

print("model_cross",model_cross)
print("model_logsoftmax",model_logsoftmax)

model_cross SimpleModel(
  (gru): GRU(5, 6)
  (linear): Linear(in_features=6, out_features=4, bias=True)
)
model_logsoftmax SimpleModel(
  (gru): GRU(5, 6)
  (linear): Linear(in_features=6, out_features=4, bias=True)
)


In [18]:
result_cross = model_cross(x)
result_logsoftmax = model_logsoftmax(x)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  


In [20]:
print("result_cross",result_cross.size())
print("result_cross",result_cross)
#2 steps
#3 batch size
#4 number of classes
#result 1 , all negative because of log

result_cross torch.Size([2, 3, 4])
result_cross tensor([[[ 0.0032,  0.1947,  0.2598, -0.4067],
         [ 0.0695,  0.1574,  0.1933, -0.4582],
         [ 0.0250,  0.1352,  0.1841, -0.4064]],

        [[ 0.0310,  0.1820,  0.2941, -0.4714],
         [ 0.0780,  0.1289,  0.1839, -0.4617],
         [ 0.0777,  0.1320,  0.1961, -0.4374]]], grad_fn=<AddBackward0>)


In [23]:
print("result_logsoftmax",result_logsoftmax.size())
print("result_logsoftmax",result_logsoftmax)
#2 steps
#3 batch size
#4 number of classes
#result 1 , all negative because of log

result_logsoftmax torch.Size([2, 3, 4])
result_logsoftmax tensor([[[-0.7413, -0.6607, -0.8037, -0.6391],
         [-0.7207, -0.6572, -0.7711, -0.6637],
         [-0.7296, -0.6717, -0.7849, -0.6433]],

        [[-0.6472, -0.7267, -0.5936, -0.7502],
         [-0.6664, -0.7305, -0.6209, -0.7234],
         [-0.6580, -0.7150, -0.6091, -0.7456]]], grad_fn=<LogSoftmaxBackward>)


In [24]:
logits_cross = result_cross.view(-1, n_classes)
logits_logsoftmax= result_logsoftmax.view(-1, n_classes)
print("logits_cross",logits_cross.size())
print("logits_logsoftmax",logits_logsoftmax.size())

logits_cross torch.Size([6, 4])
logits_logsoftmax torch.Size([6, 4])


In [25]:
print("logits_cross",logits_cross)
print("y.view(-1)",y.view(-1))
loss_cross =  cross_entropy(logits_cross, y.view(-1))
print("loss_lnn",loss_cross)

logits_cross tensor([[ 0.0032,  0.1947,  0.2598, -0.4067],
        [ 0.0695,  0.1574,  0.1933, -0.4582],
        [ 0.0250,  0.1352,  0.1841, -0.4064],
        [ 0.0310,  0.1820,  0.2941, -0.4714],
        [ 0.0780,  0.1289,  0.1839, -0.4617],
        [ 0.0777,  0.1320,  0.1961, -0.4374]], grad_fn=<ViewBackward>)
y.view(-1) tensor([2, 0, 2, 0, 0, 0])
loss_lnn tensor(7.7747, grad_fn=<NllLossBackward>)


In [26]:
print("logits_logsoftmax",logits_logsoftmax.size())
print("y.view(-1)",y.view(-1).size())
print("logits_logsoftmax",logits_logsoftmax)
print("y.view(-1)",y.view(-1))
loss_logsoftmax = nll_loss(logits_logsoftmax, y.view(-1))
print("loss_logsoftmax",loss_logsoftmax)

logits_logsoftmax torch.Size([6, 4])
y.view(-1) torch.Size([6])
logits_logsoftmax tensor([[-0.7413, -0.6607, -0.8037, -0.6391],
        [-0.7207, -0.6572, -0.7711, -0.6637],
        [-0.7296, -0.6717, -0.7849, -0.6433],
        [-0.6472, -0.7267, -0.5936, -0.7502],
        [-0.6664, -0.7305, -0.6209, -0.7234],
        [-0.6580, -0.7150, -0.6091, -0.7456]], grad_fn=<ViewBackward>)
y.view(-1) tensor([2, 0, 2, 0, 0, 0])
loss_logsoftmax tensor(4.2808, grad_fn=<NllLossBackward>)


In [27]:
for x in logits_logsoftmax[0,:]:
  print( x,exp(x))

tensor(-0.7413, grad_fn=<SelectBackward>) 0.4764848463683123
tensor(-0.6607, grad_fn=<SelectBackward>) 0.5165003973276866
tensor(-0.8037, grad_fn=<SelectBackward>) 0.4476625411977141
tensor(-0.6391, grad_fn=<SelectBackward>) 0.5277488535078471


In [44]:
#CrossEntropyLoss calculation 
#loss(x, class) = -log(exp(x[class]) / (\sum_j exp(x[j])))
#               = -x[class] + log(\sum_j exp(x[j]))

In [42]:
#formula when correct
loss = -1 + math.log(exp(0) + exp(0) + exp(0) + exp(1))
loss

0.743668380628679

In [43]:
#formula when wrong
loss = -0 + math.log(exp(1) + exp(0) + exp(0) + exp(0))
loss

1.743668380628679

In [34]:
#calculate with library
output = Variable(torch.FloatTensor([0,0,0,1])).view(1, -1)
target = Variable(torch.LongTensor([3]))

criterion = nn.CrossEntropyLoss()
loss = criterion(output, target)
print(loss)

tensor(0.7437)


In [45]:
#calculate with library

output = Variable(torch.FloatTensor([1,0,0,0])).view(1, -1)
target = Variable(torch.LongTensor([3]))

criterion = nn.CrossEntropyLoss()
loss = criterion(output, target)
print(loss)

tensor(1.7437)
